In [1]:
# Importing all the required libraries

import mysql.connector 
import pandas as pd
import boto3
from io import StringIO
import credentials as cre
import email
import smtplib
from datetime import datetime

In [2]:
# Connecting to AWS

session = boto3.Session(
    aws_access_key_id=cre.Access_key,
    aws_secret_access_key=cre.Secret_key)

In [3]:
# Creating bucket on S3 

s3 = session.resource('s3')

bucket_name = 'yu-stackoverflow-data-staging'

s3_client = boto3.client('s3', 
                         aws_access_key_id=cre.Access_key,
                         aws_secret_access_key=cre.Secret_key)
    
bucket_list = [bucket.name for bucket in s3.buckets.all()]

if bucket_name not in bucket_list:
    s3_client.create_bucket(Bucket=bucket_name)
    print('Bucket created successfully')
else: 
    print('Bucket name already exists')

Bucket created successfully


In [4]:
# Connecting to Mysql database

cnx = mysql.connector.connect(host='relational.fit.cvut.cz',
                             password='relational',
                             user='guest',
                             database='stats',use_pure=True)

cursor = cnx.cursor(buffered=True)
cursor.execute("SHOW TABLES")

my_bucket = s3.Bucket(bucket_name)

# list of Bucket objects
b_list = [my_bucket_object.key for my_bucket_object in my_bucket.objects.all()]

for table_name in cursor:
    name = table_name[0]
    if name+'.csv' not in b_list:
        sql_select_Query = "select * from "+ name
        result_dataFrame = pd.read_sql(sql_select_Query,cnx)     
        
        #loading data as CSV
        csv_buffer = StringIO()
        result_dataFrame.to_csv(csv_buffer, index = False)
        s3.Object(bucket_name, name+'.csv').put(Body=csv_buffer.getvalue())
        print(name+'.csv','file loaded successfully')
    else:
        print(name+'.csv','file already exists')
    
cnx.close()

badges.csv file loaded successfully
comments.csv file loaded successfully
postHistory.csv file loaded successfully
postLinks.csv file loaded successfully
posts.csv file loaded successfully
tags.csv file loaded successfully
users.csv file loaded successfully
votes.csv file loaded successfully


In [5]:
# Email notification if data is loaded on S3

def send_email_notification(msg):
    gmail_user = cre.gmail_user
    gmail_password = cre.gmail_password
    now = datetime.now() # current date and time

    sent_from = gmail_user
    to = ['snayak@mail.yu.edu']
    subject = 'Job Completed on: '+ now.strftime("%m/%d/%Y - %H:%M:%S")
    body = 'Task processed with 0 Errors'+ ' Task name: '+ msg

    email_text = """\
    From: %s
    To: %s
    Subject: %s 
    
    %s
    """ % (sent_from, to, subject, body)

    try:
     
        server = smtplib.SMTP('smtp.gmail.com', 587) 
        server.ehlo()
        server.starttls() 
        server.login(gmail_user, gmail_password)
        server.sendmail(sent_from, to, email_text)
        server.close()
        print ('Email successfully sent')
    except:
        print ('Error: your email did not send')

In [6]:
cnx = mysql.connector.connect(host='relational.fit.cvut.cz',
                             password='relational',
                             user='guest',
                             database='stats',use_pure=True)

cursor = cnx.cursor(buffered=True)
cursor.execute("SHOW TABLES")

my_bucket = s3.Bucket(bucket_name)

# list of Bucket objects
b_list = [my_bucket_object.key for my_bucket_object in my_bucket.objects.all()]

counter = 0
for table_name in cursor:
    name = table_name[0]
    if name+'.csv' in b_list:
        counter+=1

msg = input()
if msg == 's3 data loading':
    if counter == len(b_list): # if all the files are uploaded from mysql database 
        send_email_notification(msg)
else:
    print('give proper msg')

s3 data loading
Email successfully sent


References:
    
1. https://www.tutorialspoint.com/how-to-show-all-the-tables-present-in-the-database-and-server-in-mysql-using-python
2. https://www.edureka.co/community/31905/how-to-print-the-names-of-all-the-bucket-in-s3-in-python
3. https://www.stackvidhya.com/list-contents-of-s3-bucket-using-boto3-python/